<a href="https://colab.research.google.com/github/iyoushe1703/glassdoor-data-science-jobs-predictor/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 3000)
pd.set_option('max_rows', 1000)

In [ ]:
df = pd.read_csv("glassdoor_jobs.csv")

# Cleaning Salary Estimates Column

#### Removing missing values

In [ ]:
df = df[df['Salary Estimate'] != '-1']

In [ ]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

#### Removing "Glassdoor Estimate"

In [ ]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

#### Removing $ and K

In [ ]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.replace('$', '').replace('K', ''))

#### Creating a column to indicate 'per hour' and 'Employer Provided Salary' entries in the ```Salary Estimate``` column

In [ ]:
df['per_hour'] = df['Salary Estimate'].apply(lambda x: 1 if 'Per Hour'.lower() in x.lower() else 0)

In [ ]:
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary'.lower() in x.lower() else 0)

### Removing Employer Provided Salary from Salary Estimate

In [ ]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.lower().replace("per hour", ""))

In [ ]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.lower().replace("employer provided salary:", ""))

#### Creating ```min_salary```, ```max_salary``` and ```avg_salary``` columns

In [ ]:
df['min_salary'] = df['Salary Estimate'].apply(lambda x: x.split('-')[0])

In [ ]:
df['min_salary'] = df['min_salary'].astype(int)

In [ ]:
df['max_salary'] = df['Salary Estimate'].apply(lambda x: x.split('-')[-1])

In [ ]:
df['max_salary'] = df['max_salary'].astype(int)

In [ ]:
df["avg_salary"] = (df['min_salary'] + df['max_salary'])/2

## Converting hourly wage to annual wage

In [ ]:
# converting min_salary from hourly basis to annual basis
df['min_salary'] = df.apply(lambda x: x.min_salary * 2 if x.per_hour == 1 else x.min_salary, axis = 1)

# converting max_salary from hourly basis to annual basis
df['max_salary'] = df.apply(lambda x: x.max_salary * 2 if x.per_hour == 1 else x.max_salary, axis = 1)

In [ ]:
#df.loc[df['per_hour'] == 1, ['per_hour', 'min_salary','max_salary']]

### Cleaning ```Company Name```

In [ ]:
df['company_txt'] = df['Company Name'].apply(lambda x: x.split("\n")[0])

### Separating State from city names

In [ ]:
df['job_state'] = df['Location'].apply(lambda x: x[-2:])

### Make a new column that indicates whether the job location is at the company headquarters

#### ```np.select``` is more efficient than ```.apply()```

In [ ]:
import numpy as np

condition = [df['Location'] == df['Headquarters'],
             df['Location'] != df['Headquarters']]

outputs = [1, 0]

df['job_at_hq'] = np.select(condition, outputs)

# Company age


In [ ]:
df['age'] = df['Founded'].apply(lambda x: x if x < 1 else 2020 - x)

# Parsing Job Description

In [ ]:
# checking if python is in the job description
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [ ]:
# checking if R is in the job description
df['rstudio'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() else 0)

In [ ]:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [ ]:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [ ]:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

# Simplifying data science job titles

In [ ]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'
    
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'jr'
    else:
        return 'na'

In [ ]:
df['job_simp'] = df['Job Title'].apply(title_simplifier)

In [ ]:
df['seniority'] = df['Job Title'].apply(seniority)

# Miscellaneous 

### Column to indicate the length of the description

In [ ]:
df['desc_len'] = df['Job Description'].apply(lambda x: len(x))

### Column to indicate number of competitors

In [ ]:
df['competitors_count'] = df['Competitors'].apply(lambda x: x.split(',') if x != "-1" else 0)

# Export cleaned ```df``` to a .csv file

In [ ]:
df.to_csv("glassdoor_cleaned.csv", index = False)